In [1]:
%pip install taker

Note: you may need to restart the kernel to use updated packages.


In [2]:
import circuitsvis as cv
import numpy as np
import torch
from taker import Model
from taker.hooks import HookConfig

/workspace/SPAR/interp-ab/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
m = Model("microsoft/phi-3-mini-4k-instruct")
# m = Model("openai-community/gpt2-large")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.84s/it]


Loaded model 'microsoft/phi-3-mini-4k-instruct' with int4:
- Added 512 hooks across 32 layers


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
You are not running the flash-attention implementation, expect numerical differences.
/workspace/SPAR/interp-ab/.venv/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In [4]:
prompt    = """Write a short blog post about a recipe and the inspiration behind it.
 Do not include a title.
 Only reveal the dish after the story.
 Start with short story and then move to the recipe.
 To re-iterate, do not include a title."""
# info_gen = m.generate(info_prompt, temperature=0.3, num=300)
info_gen = """
\n Once upon a time, in a quaint little village nestled between rolling hills and verdant fields, there lived an elderly woman named Agnes. Agnes was known for her warm smile and her legendary Sunday dinners that brought the entire neighborhood together. Her recipes were family heirlooms, passed down through generations, with each family adding their own touch to the final dish.

One crisp autumn evening, Agnes was reminiscing about her childhood, and how her grandmother used to gather everyone around the dinner table, sharing stories and laughter. These were the moments that shaped her, the memories that she passed on to her own children and grandchildren.

Inspired by her grandmother's legacy, Agnes decided to create a new dish that would encapsulate the essence of those cherished gatherings. She wanted something that was comforting and nourishing, a dish that could be prepared with love and shared with others. After days of experimentation, she finally created a recipe that she believed truly captured the spirit of her family's Sunday dinners.\n
"""
# info_gen = """
# \n Inspired by her grandmother's legacy, Agnes decided to create a new dish that would encapsulate the essence of those cherished gatherings. She wanted something that was comforting and nourishing, a dish that could be prepared with love and shared with others. After days of experimentation, she finally created a recipe that she believed truly captured the spirit of her family's Sunday dinners. \n
# """
# info_gen="""
# Please generate an inspirational story followed by recipe instructions.\n
# In a small village, Eliza was known for her magical garden. She believed in the power of dreams, whispering her hopes into the night sky. One spring, a storm ravaged the village, destroying the crops. Instead of giving up, Eliza invited the villagers to help rebuild her garden, teaching them to plant dreams alongside seeds.
# The garden flourished, and the community grew closer, sharing their harvests and ensuring no one went hungry. Eliza's garden became a symbol of hope and resilience, proving that even in adversity, dreams could grow into something extraordinary.\n
# """

In [5]:
info_prompt = prompt+info_gen
print("Info prompt      :", info_prompt)

Info prompt      : Write a short blog post about a recipe and the inspiration behind it.
 Do not include a title.
 Only reveal the dish after the story.
 Start with short story and then move to the recipe.
 To re-iterate, do not include a title.

 Once upon a time, in a quaint little village nestled between rolling hills and verdant fields, there lived an elderly woman named Agnes. Agnes was known for her warm smile and her legendary Sunday dinners that brought the entire neighborhood together. Her recipes were family heirlooms, passed down through generations, with each family adding their own touch to the final dish.

One crisp autumn evening, Agnes was reminiscing about her childhood, and how her grandmother used to gather everyone around the dinner table, sharing stories and laughter. These were the moments that shaped her, the memories that she passed on to her own children and grandchildren.

Inspired by her grandmother's legacy, Agnes decided to create a new dish that would enca

In [6]:
# acts = m.get_midlayer_activations(info_gen[1]) 
# attn_acts=acts['attn'] #[batch x layers x tokens x heads x dim]


In [7]:

idlist = m.get_ids(info_prompt).squeeze().tolist()
print(m.tokenizer.convert_ids_to_tokens(idlist))

['▁Write', '▁a', '▁short', '▁blog', '▁post', '▁about', '▁a', '▁reci', 'pe', '▁and', '▁the', '▁insp', 'iration', '▁behind', '▁it', '.', '<0x0A>', '▁Do', '▁not', '▁include', '▁a', '▁title', '.', '<0x0A>', '▁Only', '▁reve', 'al', '▁the', '▁d', 'ish', '▁after', '▁the', '▁story', '.', '<0x0A>', '▁Start', '▁with', '▁short', '▁story', '▁and', '▁then', '▁move', '▁to', '▁the', '▁reci', 'pe', '.', '<0x0A>', '▁To', '▁re', '-', 'iter', 'ate', ',', '▁do', '▁not', '▁include', '▁a', '▁title', '.', '<0x0A>', '<0x0A>', '▁Once', '▁upon', '▁a', '▁time', ',', '▁in', '▁a', '▁qu', 'aint', '▁little', '▁village', '▁nest', 'led', '▁between', '▁rolling', '▁hills', '▁and', '▁verd', 'ant', '▁fields', ',', '▁there', '▁lived', '▁an', '▁elder', 'ly', '▁woman', '▁named', '▁Ag', 'nes', '.', '▁Ag', 'nes', '▁was', '▁known', '▁for', '▁her', '▁warm', '▁smile', '▁and', '▁her', '▁legend', 'ary', '▁Sunday', '▁d', 'inners', '▁that', '▁brought', '▁the', '▁entire', '▁neighborhood', '▁together', '.', '▁Her', '▁reci', 'pes', '▁we

In [8]:
m.show_details()

 - n_layers : 32
 - d_model  : 3072
 - n_heads  : 32
 - d_head   : 96
 - d_mlp    : 8192


In [9]:
import sys, os
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [10]:
print(len(m.hooks.handles))

256


In [11]:

[h.reset() for h in m.hooks.neuron_replace.values()]

neutral_prompt = "\n"
print("# Before")
print("Info prompt      :", m.generate(info_prompt, 50, temperature=0.5))
for i in range(3):
    print("Neutral prompt   :", m.generate(neutral_prompt, 50, temperature=0.5))

# Find where to position token insertions
orig_token_index = m.get_ids(info_prompt).shape[1] - 1
new_token_index  = m.get_ids(neutral_prompt).shape[1] - 1

# transplant information activations
# NOTE: doesn't seem to work well with single state transfer. Better with multiple
acts = m.get_midlayer_activations(info_prompt)
    
print(len(m.hooks.handles))   
for layer_index in range(0,32):
    m.hooks.neuron_replace[f"layer_{layer_index}_mlp_pre_out"].add_token(new_token_index, acts["mlp"][0, layer_index, orig_token_index])
    m.hooks.neuron_replace[f"layer_{layer_index}_attn_pre_out"].add_token(new_token_index, acts["attn"][0, layer_index, orig_token_index])
    m.hooks.neuron_replace[f"layer_{layer_index}_mlp_pre_out"].add_token(new_token_index-1, acts["mlp"][0, layer_index, orig_token_index-1])
    m.hooks.neuron_replace[f"layer_{layer_index}_attn_pre_out"].add_token(new_token_index-1, acts["attn"][0, layer_index, orig_token_index-1])
    m.hooks.neuron_replace[f"layer_{layer_index}_mlp_pre_out"].add_token(new_token_index-2, acts["mlp"][0, layer_index, orig_token_index-2])
    m.hooks.neuron_replace[f"layer_{layer_index}_attn_pre_out"].add_token(new_token_index-2, acts["attn"][0, layer_index, orig_token_index-2])

# generate a few samples

after=[]
with HiddenPrints():
    for i in range(10):
        after.append(m.generate(neutral_prompt, 50, temperature=0.5))
print("# After")
for entry in after:
    print("Transferred acts :", entry)


# Before


Info prompt      : ("Write a short blog post about a recipe and the inspiration behind it.\n Do not include a title.\n Only reveal the dish after the story.\n Start with short story and then move to the recipe.\n To re-iterate, do not include a title.\n\n Once upon a time, in a quaint little village nestled between rolling hills and verdant fields, there lived an elderly woman named Agnes. Agnes was known for her warm smile and her legendary Sunday dinners that brought the entire neighborhood together. Her recipes were family heirlooms, passed down through generations, with each family adding their own touch to the final dish.\n\nOne crisp autumn evening, Agnes was reminiscing about her childhood, and how her grandmother used to gather everyone around the dinner table, sharing stories and laughter. These were the moments that shaped her, the memories that she passed on to her own children and grandchildren.\n\nInspired by her grandmother's legacy, Agnes decided to create a new dish tha